In [19]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
import json
import nltk
import unicodedata
import re
import string
import joblib as joblib
import pickle as pickle
from sklearn.model_selection import cross_val_score
# nltk.download('punkt')
# pip install unidecode
from unidecode import unidecode

In [20]:
# Load data
data_train = pd.read_csv('../Dataset/train_data.csv')
data_test = pd.read_csv('../Dataset/test_data.csv')

In [21]:
X_train = data_train['article']
y_train = data_train['label']

X_test = data_test['article']
y_test = data_test['label']

In [22]:
# Text preprocessing
def wordopt(text):
    text = text.lower()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

ps = PorterStemmer()
with open('../Resources/stopwords-tl.json', 'r') as f:
    stopwords = json.load(f)

def preprocess(text):
    text = wordopt(text)
    tokens = nltk.word_tokenize(text)
    stems = [ps.stem(token) for token in tokens]
    filtered = [stem for stem in stems if stem not in stopwords]
    return filtered

In [23]:
# Apply preprocess function to X_train
X_train = X_train.apply(preprocess)

# Apply preprocess function to X_test
X_test = X_test.apply(preprocess)

In [ ]:
# Convert the preprocessed text into a list of strings
X_train_processed = [' '.join(tokens) for tokens in X_train]
X_test_processed = [' '.join(tokens) for tokens in X_test]

# Initialize the TfidfVectorizer
tfidf_vectorizer = joblib.load('../Models/Vectorizer/vectorizer_lr.joblib')

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.transform(X_train_processed)

# Transform the validation data using the same vectorizer
X_test_tfidf = tfidf_vectorizer.transform(X_test_processed)

# Print the shape of the TF-IDF matrices (No of rows, No of columns)
print("Shape of X_train_tfidf:", X_train_tfidf.shape)
print("Shape of X_test_tfidf:", X_test_tfidf.shape)

Shape of X_train_tfidf: (2103, 27707)
Shape of X_test_tfidf: (902, 27707)


In [ ]:
mnb = joblib.load('../Models/Base/basemodel_MNB_grid.joblib').best_estimator_
lr = joblib.load('../Models/Base/basemodel_LR_grid.joblib').best_estimator_
rf = joblib.load('../Models/Base/basemodel_RF_grid.joblib').best_estimator_
knn = joblib.load('../Models/Base/basemodel_KNN_grid.joblib').best_estimator_
svm = joblib.load('../Models/Base/basemodel_SVM_grid.joblib').best_estimator_
mlp = joblib.load('../Models/Grid/grid_mlp_2.joblib').best_estimator_

In [ ]:
base_estimators = [
    ('mnb',mnb),
    ('lr',lr),
    ('rf',rf),
    ('knn',knn),
    ('svm',svm)
]

In [ ]:
# Create StackingClassifier
stack = StackingClassifier(estimators=base_estimators, final_estimator=lr, cv='prefit')

# Fit StackingClassifier
stack.fit(X_train_tfidf, y_train)

KeyboardInterrupt: 

In [ ]:
# Evaluate stacking model
stack_y_pred = stack.predict(X_test_tfidf)
stack_y_prob = stack.predict_proba(X_test_tfidf)[:, 1]  # Probability for positive class for ROC curve

stack_accuracy = accuracy_score(y_test, stack_y_pred)
stack_precision = precision_score(y_test, stack_y_pred)
stack_recall = recall_score(y_test, stack_y_pred)
stack_f1 = f1_score(y_test, stack_y_pred)
stack_auc = roc_auc_score(y_test, stack_y_prob)

print("Metrics for Stacking Model:")
print(f"Accuracy: {stack_accuracy:.4f}")
print(f"Precision: {stack_precision:.4f}")
print(f"Recall: {stack_recall:.4f}")
print(f"F1 Score: {stack_f1}:.4f")
print(f"AUC: {stack_auc:.4f}")
print(f"Classification Report:\n{classification_report(y_test, stack_y_pred)}")

# ROC Curve for stacking model
stack_fpr, stack_tpr, _ = roc_curve(y_test, stack_y_prob)
stack_roc_auc = roc_auc_score(y_test, stack_y_prob)

# Plot ROC Curve for stacking model
plt.figure(figsize=(8, 6))
plt.plot(stack_fpr, stack_tpr, label=f'RF (AUC = {stack_roc_auc:.4f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Stacking Model')
plt.legend(loc="lower right")
plt.show()